# To SPEEDY Model Grid Interpoler

## Project's Folder Structure

```
SPEEDY-Interpoler/
├── Data/
│   ├── Interpolations/
│   │   └── All resulting interpolations goes  here 
│   ├── NOAA/
│   │   ├── Atmospherical_Conditions/
│   │   │   └── NOAA NETCDF Files of Atmospherical Conditions
│   │   └── Pressure_Conditions/
│   │       └── NOAA NETCDF Files relative of Pressure on Surface information
│   └── SPEEDY/
│       └── SPEEDY model .grd Files
└──Notebooks/
    └── Required Jupyter Notebooks to perform interpolation and error estimation
```

## Imports and Path Settings

In [1]:
import struct
from pathlib import Path

import numpy as np
import xarray as xr
from scipy.interpolate import griddata
from netCDF4 import Dataset

In [2]:
PATH = Path.cwd()/'../Data/NOAA/Atmospherical_Conditions'
PRESSURE_PATH = Path.cwd()/'../Data/NOAA/Pressure_Conditions'
INTERPOLATIONS_PATH = Path.cwd()/'../Data/Interpolations'

IS_SPEEDY_ORDER = True
# IS_SPEEDY_ORDER determines if varibles should follow SPEEDY model variables order. 

if (IS_SPEEDY_ORDER):
    FILES = ['uwnd.2020.nc','vwnd.2020.nc','air.2020.nc','rhum.2020.nc']
else:
    FILES = [k.name for k in PATH.rglob('*.nc')]
    list.sort(FILES)

## Model Settings

In [3]:
# NOAA Default pressure leves
PRESSURE_LEVELS_VALUES = [925,850,700,500,300,200,100]

# Grids definition for SPEEDY and NOAA
X_speedy_long = np.linspace(0,360-3.75,96)
Y_speedy_lat = np.array("-87.159 -83.479 -79.777 -76.070 -72.362 -68.652 -64.942 -61.232 -57.521 -53.810 -50.099 -46.389 -42.678 -38.967 -35.256 -31.545 -27.833 -24.122 -20.411 -16.700 -12.989 -9.278 -5.567 -1.856 1.856 5.567 9.278 12.989 16.700 20.411 24.122 27.833 31.545 35.256 38.967 42.678 46.389 50.099 53.810 57.521 61.232 64.942 68.652 72.362 76.070 79.777 83.479 87.159".split(" "))
Y_speedy_lat = Y_speedy_lat.astype(np.float32)
#Y_speedy_lat = np.flipud(Y_speedy_lat)

X_noaa_long = np.linspace(0,360-2.5,144)
Y_noaa_lat = np.linspace(-90,90,73)
X_grid_noaa, Y_grid_noaa = np.meshgrid(X_noaa_long,Y_noaa_lat)


SPEEDY_LON = 96;
SPEEDY_LAT = 48;
SPEEDY_LVL = 7;

INTERPOLATION_VARIABLES = len(FILES)

# Temporal setting
DATE = '2020-07-01'
TIME = '00:00:00'
## Time is HH:MM:SS in 24-hours format
DATETIME = DATE + 'T' + TIME
FILENAME = DATE.replace('-','') + TIME[:2]

IS_CONVERTION_REQUIRED = True
# IS_CONVERTION_REQUIRED performs Relative humidity convertion to Specific Humidity, if True. 
# If not, Relative Humidity is given

SAVE_AS_GRD = True
# If SAVE_AS_GRD is True, it will convert data into GRD format aditionally to the netCDF files created. 
# The atmospherical variables are in one netCDF file, and the pressure will be on another file.

## Functions

In [4]:
def read_data(variable,file):
    '''
        input variables
            file          : name of the variable
        output variable
            variable_array: 3-dimensional xarray with all the pressure levels of the given variable
    '''
    variable_path = PATH/file
    variable_array = xr.open_dataset(variable_path)[variable].sel(
        level = PRESSURE_LEVELS_VALUES,
        time = DATETIME)
    return variable_array

def interpoler(variable_grid, method = 'nearest'):
    '''
        input variables:
            variable_grid : variable in specific level to be spatially interpolated
            method        : method to use for the interpolation function (linear, nearest, cubic)
        output variable
            interp_noaa_speedy: speedy grid for certain level
    '''
    interp_noaa_speedy = griddata((X_grid_noaa.ravel(),Y_grid_noaa.ravel()), variable_grid.ravel()
                              , (X_speedy_long[None,:], Y_speedy_lat[:,None]), method=method)
    return interp_noaa_speedy

def relative2specific(T,RH,p):
    '''
    Parameters
    ----------
        T : Temperature in K.
        RH : Relative humidity in percentage [0,100].
        p : Preassure in mbar.

    Returns
    -------
        specific humidity (dimensionless)

    '''  
    T-=273.15
    p*=100
    RH/=100
    e_s=611.21*np.exp((18.687-T/234.5)*(T/(T+257.14)))
    e=e_s*RH
    w=287.058/461.5*e/(p-e)
    return w/(w+1)

## Interpolation

**Variables order in list**  
0 : Air Temperature (7 lvls)  
1 : RHumidity (7 lvls)  
2 : Uwnd (7 lvls)  
3 : Vwnd (7 lvls)  
4 : Pressure (1 lvls)  
5 : Specific Humidity (7 lvls)

### Atmospherical Conditions interpolation

In [5]:
atmospherical_variables = dict()

for file in FILES:
    variable = file.split(".")[0]
    variable_values_by_level = np.zeros((SPEEDY_LVL,SPEEDY_LAT,SPEEDY_LON))
    
    variable_array = read_data(variable,file)
    
    for index_pressure_level, pressure in enumerate(PRESSURE_LEVELS_VALUES):
        variable_values_by_level[index_pressure_level,:,:] = interpoler(variable_array.sel(level=pressure).values)
        
    atmospherical_variables[variable] = variable_values_by_level

### Relative Humidity to Specific Humidity Convertion

In [6]:
if (IS_CONVERTION_REQUIRED):
    specific_humidity_values =  np.zeros((SPEEDY_LVL,SPEEDY_LAT,SPEEDY_LON))

    for index,pressure_value in enumerate(PRESSURE_LEVELS_VALUES):
        RH_data = atmospherical_variables['rhum'][index,:,:].copy()
        air_T_data = atmospherical_variables['air'][index,:,:].copy()
        specific_humidity_values[index,:,:] = np.vectorize(relative2specific)(air_T_data,
                                                                          RH_data,
                                                                          pressure_value)

    atmospherical_variables['shum'] = specific_humidity_values
    atmospherical_variables.pop('rhum',None);
    # pop deletes the values in the dict which are contained in the Key. If no key is found, None is returned. 
    # This is done, as the rhum variable is no longer needed in the process. 
    # To check funcionality, please create a copy of dict at this point.

### Pressure interpolation

In [7]:
variable_name = 'pres'
pressure_array = xr.open_dataset(PRESSURE_PATH/'pres.sfc.2020.nc')[variable_name].sel(time = DATETIME)
pressure_values = interpoler(pressure_array.values)

atmospherical_variables[variable_name] = pressure_values

## Sanity checks

In [8]:
for key in atmospherical_variables:
    print(key)
    print(f'The Max is {atmospherical_variables[key].max()}')
    print(f'The Min is {atmospherical_variables[key].min()}')
    print('------------------------------------------------')

uwnd
The Max is 67.69999694824219
The Min is -36.599998474121094
------------------------------------------------
vwnd
The Max is 45.400001525878906
The Min is -39.900001525878906
------------------------------------------------
air
The Max is 311.5
The Min is 190.6999969482422
------------------------------------------------
shum
The Max is 0.02340727900319144
The Min is 0.0
------------------------------------------------
pres
The Max is 105150.0
The Min is 50900.0
------------------------------------------------


## Convert to GRD vector form

In [9]:
atmospherical_variables.keys()

dict_keys(['uwnd', 'vwnd', 'air', 'shum', 'pres'])

In [10]:
if(SAVE_AS_GRD):
    result_list = list()
    for variable in atmospherical_variables.values():
        result_list.extend(variable.ravel().tolist())
    fout = open(INTERPOLATIONS_PATH/(FILENAME+'.grd'), 'wb')
    for i in result_list:
        fout.write(struct.pack('>f',i))
    fout.close()

In [11]:
example = np.array([[[1,2,3],[2,6,4]],[[8,9,17],[32,64,12]]])
example

array([[[ 1,  2,  3],
        [ 2,  6,  4]],

       [[ 8,  9, 17],
        [32, 64, 12]]])

In [12]:
a = example.ravel(order='A')
a

array([ 1,  2,  3,  2,  6,  4,  8,  9, 17, 32, 64, 12])

In [13]:
l = 0
exa = np.zeros((2,2,3))
for i in range(2):
    for j in range(2):
        for k in range(3):
            exa[i,j,k] = a[l]
            l +=1

In [14]:
exa

array([[[ 1.,  2.,  3.],
        [ 2.,  6.,  4.]],

       [[ 8.,  9., 17.],
        [32., 64., 12.]]])

In [15]:
pressure = atmospherical_variables.pop('pres',None);

atmospherical_variables_to_netcdf = dict()
pressure_to_netcdf = dict()

for key, value in atmospherical_variables.items():
    atmospherical_variables_to_netcdf[key] = (("level","lat", "lon"),value)

pressure_to_netcdf['pres'] = (("lat", "lon"),pressure)

In [16]:
atmospherical_dataset = xr.Dataset(
    atmospherical_variables_to_netcdf
    ,coords={
        "level": PRESSURE_LEVELS_VALUES,
        "lat": Y_speedy_lat,
        "lon": X_speedy_long,
    },
    attrs = {
        'long_name': '6-Hourly Sample',
        'Levels': 7,
        'dataset': 'NCEP/DOE AMIP-II Reanalysis (Reanalysis-2)',
        'level_desc': 'Surface',
        'statistic': 'Individual Obs',
    },
)

In [17]:
pressure_dataset = xr.Dataset(
    pressure_to_netcdf
    ,coords = {
        "lat": Y_speedy_lat,
        "lon": X_speedy_long,
    },
    attrs = {
        'long_name': '6-Hourly Pressure at Surface',
        'Levels': 1,
        'units': 'Pascals',
        'precision': -1,
        'GRIB_id': 1,
        'GRIB_name': 'PRES',
        'var_desc': 'Pressure',
        'dataset': 'NCEP/DOE AMIP-II Reanalysis (Reanalysis-2)',
        'level_desc': 'Surface',
        'statistic': 'Individual Obs',
        'parent_stat': 'Other',
        'standard_name': 'pressure',  
    },
)

In [18]:
atmospherical_dataset

<xarray.Dataset>
Dimensions:  (lat: 48, level: 7, lon: 96)
Coordinates:
  * level    (level) int32 925 850 700 500 300 200 100
  * lat      (lat) float32 -87.159 -83.479 -79.777 ... 79.777 83.479 87.159
  * lon      (lon) float64 0.0 3.75 7.5 11.25 15.0 ... 345.0 348.8 352.5 356.2
Data variables:
    uwnd     (level, lat, lon) float64 -6.9 -6.5 -5.6 -5.1 ... -3.2 -3.6 -3.8
    vwnd     (level, lat, lon) float64 -5.6 -5.8 -6.0 -6.0 ... -2.8 -2.5 -2.3
    air      (level, lat, lon) float64 247.1 247.4 247.8 ... 229.0 229.0 229.0
    shum     (level, lat, lon) float64 0.0003112 0.0002995 ... 7.599e-06
Attributes:
    long_name:   6-Hourly Sample
    Levels:      7
    dataset:     NCEP/DOE AMIP-II Reanalysis (Reanalysis-2)
    level_desc:  Surface
    statistic:   Individual Obs

In [19]:
atmospherical_dataset.to_netcdf(INTERPOLATIONS_PATH/(FILENAME + "-atmospherical_dataset.nc"))